### Importación de librerias

In [1]:
from re import M
import pandas as pd
import numpy as np

import xml.etree.ElementTree as ET
import os
from gensim.parsing.porter import PorterStemmer 
from nltk.corpus import stopwords
import json

In [2]:
def get_documents(path):
    documents = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith(".naf"):
            tree = ET.parse(path+filename)
            texto = tree.find('raw').text
            documents.append(texto)
    return documents

In [3]:
datos = get_documents('datos/docs-raw-texts/')
documentos = pd.DataFrame(datos,columns=['Documento'])
documentos.head()

,Documento
0,William Beaumont and the Human Digestion.\n\nW...
1,Selma Lagerlöf and the wonderful Adventures of...
2,Ferdinand de Lesseps and the Suez Canal.\n\nFe...
3,Walt Disney’s ‘Steamboat Willie’ and the Rise ...
4,Eugene Wigner and the Structure of the Atomic ...


In [4]:
def remove_stopwords(data):
    filtered_words = [word.lower() for word in data.split(' ') if word.lower() not in stopwords.words('english')]    
    return filtered_words

In [5]:
def preprocesar(documentos):
# 2. Preprocess the data
#remover espacios dobles y triples
    import re
    documentos = re.sub('\n', ' ',documentos)
    documentos = re.sub('[^a-zA-Z]|[0-9]', ' ',documentos)
    documentos = re.sub('\s+', ' ',documentos)
    p=PorterStemmer()
    documentos = p.stem_sentence(documentos)

    filtrada= remove_stopwords(documentos)
    
    return filtrada

In [6]:
documentos['filtrada']=documentos['Documento'].apply(preprocesar)
doc_proc= documentos
doc_proc.filtrada = doc_proc.filtrada.apply(np.unique)
doc_proc.head()

,Documento,filtrada
0,William Beaumont and the Human Digestion.\n\nW...,"[accid, acid, activ, affect, ag, alexi, also, ..."
1,Selma Lagerlöf and the wonderful Adventures of...,"[abl, academi, accept, acclaim, accomplish, ac..."
2,Ferdinand de Lesseps and the Suez Canal.\n\nFe...,"[abandon, act, adopt, affair, africa, afterwar..."
3,Walt Disney’s ‘Steamboat Willie’ and the Rise ...,"[aboard, accident, accompani, ad, along, also,..."
4,Eugene Wigner and the Structure of the Atomic ...,"[accept, achiev, ad, administr, albert, along,..."


In [7]:
indice_invertido = {}
for i in range(len(doc_proc)):
    for j in range(len(doc_proc.iloc[i]['filtrada'])):
        if doc_proc.iloc[i]['filtrada'][j] not in indice_invertido:
            indice_invertido[doc_proc.iloc[i]['filtrada'][j]] = []
        indice_invertido[doc_proc.iloc[i]['filtrada'][j]].append(i+1)

In [8]:
a_file = open("salida/indice_invertido.json", "w")
json.dump(indice_invertido, a_file)
a_file.close()

In [10]:
a_file = open("salida/indice_invertido.json", "r")
indice_invertido = json.loads(a_file.read())
datos_querry = get_documents('datos/queries-raw-texts/')
queries = pd.DataFrame(datos_querry,columns=['Query'])
queries


,Query
0,Fabrication of music instruments
1,famous German poetry
2,Romanticism
3,University of Edinburgh research
4,bridge construction
5,Walk of Fame stars
6,Scientists who worked on the atomic bomb
7,Invention of the Internet
8,early telecommunication methods
9,Who explored the South Pole


In [11]:
queries['filtrada'] = queries.Query.apply(preprocesar)
quer_proc = queries
quer_proc.filtrada = quer_proc.filtrada.apply(np.unique)
quer_proc.head()

,Query,filtrada
0,Fabrication of music instruments,"[fabric, instrument, music]"
1,famous German poetry,"[famou, german, poetri]"
2,Romanticism,[romantic]
3,University of Edinburgh research,"[edinburgh, research, univers]"
4,bridge construction,"[bridg, construct]"


In [12]:
def BSII_AND(lista_busqueda):
    if len(lista_busqueda) == 0:
        return ''
    if len(lista_busqueda) == 1:
        listaA = indice_invertido[lista_busqueda[0]]
        return ','.join([f'd{x:03}' for x in listaA])
    else:
        listaA = indice_invertido[lista_busqueda[0]].copy()
        for i in range(1,len(lista_busqueda)):
            if lista_busqueda[i] in indice_invertido:
                listaB = indice_invertido[lista_busqueda[i]]
                listaC = []
                for elem in listaA:
                    if elem in listaB:
                        listaC.append(elem)
                listaA = listaC.copy()
            else:
                return ''
        
        return ','.join([f'd{x:03}' for x in listaA])

In [13]:
q=quer_proc.filtrada.apply(BSII_AND)
f = open("salida/BSII-AND-queries_results.txt", "w")
for i in range(len(q)):
    f.write(f'q{i+1:02} {q[i]}\n')
f.close()

In [14]:
def BSII_OR(lista_busqueda):
    if len(lista_busqueda) == 0:
        return ''
    if len(lista_busqueda) == 1:
        listaA = indice_invertido[lista_busqueda[0]]
        return ','.join([f'd{x:03}' for x in listaA])
    else:
        listaA = indice_invertido[lista_busqueda[0]].copy()
        for i in range(1,len(lista_busqueda)):
            if lista_busqueda[i] in indice_invertido:
                listaB = indice_invertido[lista_busqueda[i]]
                listaC =  listaA.copy()+listaB.copy()
                listaA = listaC.copy()
            else:
                return ''
        listaA = np.unique(listaA)        
        return ','.join([f'd{x:03}' for x in listaA])

In [15]:
q_or=quer_proc.filtrada.apply(BSII_OR)
f = open("salida/BSII-OR-queries_results.txt", "w")
for i in range(len(q_or)):
    f.write(f'q{i+1:02} {q_or[i]}\n')
f.close()